In [238]:
# Written bu: Brian Dorricott
# (c) Copright, Brian Dorricott, 2021
#
import pandas as pd
import glob

In [239]:
# Let's read ain the accident files. Note name format changes between years
#
# Make sure the accident files are in the subdirectory /Data
# Simply drop files into the directory to have them included in the analysis
files = glob.glob("Data\*Accidents*.csv")
df = pd.DataFrame()
for file in files:
    nx = pd.read_csv(file,low_memory=False)
    df = df.append(nx, ignore_index=True)

# Setting up the index makes a significant speed improvement to processing later
df["index"] = df["Accident_Index"]
df.set_index('index', inplace=True)

print("Read ", df.shape[0], " records")

Read  240171  records


In [240]:
# Let's look at some
df.sample(10)

,Accident_Index,Location_Easting_OSGR,Location_Northing_OSGR,Longitude,Latitude,Police_Force,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Date,...,Pedestrian_Crossing-Human_Control,Pedestrian_Crossing-Physical_Facilities,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Special_Conditions_at_Site,Carriageway_Hazards,Urban_or_Rural_Area,Did_Police_Officer_Attend_Scene_of_Accident,LSOA_of_Accident_Location
index,,,,,,,,,,,,,,,,,,,,,
2019220893220,2019220893220,381716.0,273192.0,-2.269884,52.356495,22,3,1,1,24/10/2019,...,0,0,4,1,1,0,0,2,1,E01032459
20191365J0504,20191365J0504,411256.0,438149.0,-1.830430,53.839486,13,3,2,1,19/05/2019,...,0,0,1,1,1,0,0,1,1,E01010596
2019331900215,2019331900215,472347.0,288716.0,-0.935877,52.491585,33,2,2,2,01/03/2019,...,0,0,1,1,2,7,0,2,1,E01025794
2018360821021,2018360821021,610522.0,324640.0,1.119889,52.778302,36,3,2,1,29/09/2018,...,0,0,1,1,1,0,0,2,2,E01026523
2018521805916,2018521805916,340686.0,169993.0,-2.854504,51.425824,52,3,2,1,07/08/2018,...,0,0,1,1,1,0,0,1,1,E01014763
2018400310445,2018400310445,508580.0,221526.0,-0.423901,51.881743,40,3,1,1,05/07/2018,...,0,0,1,1,1,0,0,1,1,E01015729
2019010213342,2019010213342,537662.0,185958.0,-0.015675,51.555719,1,3,2,1,22/10/2019,...,0,0,1,1,1,0,0,1,1,E01004430
2018350259133,2018350259133,547278.0,259293.0,0.154207,52.212190,35,3,2,1,13/01/2018,...,0,0,1,1,1,0,0,1,1,E01017945
2018331801381,2018331801381,449538.0,316099.0,-1.267686,52.740277,33,3,2,1,07/12/2018,...,0,0,1,1,1,0,0,2,1,E01025934


In [241]:
# What data types did we get?
df.dtypes

Accident_Index                                  object
Location_Easting_OSGR                          float64
Location_Northing_OSGR                         float64
Longitude                                      float64
Latitude                                       float64
Police_Force                                     int64
Accident_Severity                                int64
Number_of_Vehicles                               int64
Number_of_Casualties                             int64
Date                                            object
Day_of_Week                                      int64
Time                                            object
Local_Authority_(District)                       int64
Local_Authority_(Highway)                       object
1st_Road_Class                                   int64
1st_Road_Number                                  int64
Road_Type                                        int64
Speed_limit                                      int64
Junction_D

In [242]:
# Minutes can be hard to process, lets create new values for PowerBI later
df["Minutes"] = df["Time"].apply(lambda x: int(str(x)[0:2])*60.0+int(str(x)[3:5]) if (x==x) else 0)
df["MinuteSlice"] = df["Minutes"].apply(lambda x: int(x/30))
#print(df[["Time", "Minutes", "MinuteSlice"]])

In [243]:
# Let's find the hotspots
#
# Standard unsupervised machine learning clustering techniques were attempted but most
# require the number of accidents hotspots to be known ahead of time and the remainder
# required a huge quantity of memory >200GB. So we've written our own taking advantage
# of special features of the dataset.
# 
# We will step through every accident to discover if there any others within 10m.
#    If so, we will check to see if any of these accidents are already part of a cluster.
#       If they are part of a cluster, use that cluster ID otherwise use a new one.
#       Mark all our accidents as part of the cluster.
# 
# This parameter determines how close accidents must be to each other to be considered to be
# in the same hotspot. It is in 10's of metres. Thus:
# 1 = wtihin 10m of each other
# 5 = within 50m of each other
# We will use 50m since this is the figure used in data for distance to juctions.
#
spot_size = 5

hotspot_id = 1
df["Hotspot_Index"] = 0
count = 0
for i in range(df.shape[0]):
#for i in range(10000):
    count += 1
    r = df.iloc[i]
    if (r["Hotspot_Index"] == 0):
        x = r["Location_Easting_OSGR"]
        y = r["Location_Northing_OSGR"]
        # Discover which accidents are within "spot_size"m away from us
        hs = df[(((df["Location_Easting_OSGR"]-x)*(df["Location_Easting_OSGR"]-x) + 
                    (df["Location_Northing_OSGR"]-y)*(df["Location_Northing_OSGR"]-y))< spot_size*spot_size)]
        if (hs.shape[0] > 1):
            # We have found two or more accidents close to each other.
            # Some might already be in a cluster so we should join them
            our_hotspot_id = hotspot_id
            hits = ""
            for j, row in hs.iterrows():
                if (row["Hotspot_Index"] != 0):
                    our_hotspot_id = row["Hotspot_Index"]
                    hits += str(row["Hotspot_Index"]) + ","

            # Note how big this cluster is before we start
            start = df[(df['Hotspot_Index'] == our_hotspot_id)].shape[0]
            # Add all accidents to the cluster
            for j, row in hs.iterrows():
                df.at[j, "Hotspot_Index"] = our_hotspot_id
            # Note how big this cluster is now
            end = df[(df['Hotspot_Index'] == our_hotspot_id)].shape[0]
            # If the cluster size has changed, we need to output what we did
            #
            # This is for checking, here's an example:
            #     Cluster of 5 with 7 accidents clashes - with 2835,2835,2835,2835, so now 8 accidents
            # THhis means that we've found a cluster of 5 accidents. One or more was already 
            # in a cluster (id 2835) which has 7 accidents. Once we've added our new accident(s)
            # we now how a total cluster size of 8 accidents.
            #
            # The important part of this is the "with" clause. Should any of these be different it
            # means that two clusters need to be combined into one. Since this never happeneed, no
            # code has been written to implement this.
            if ((hits != "") & (start != end)):
                print("Cluster of", hs.shape[0], "with",start, "accidents clashes - with", hits, "so now", end, "accidents")
            if (hotspot_id == our_hotspot_id):
                hotspot_id += 1
    # Keep human happy since this can take over 30 minutes to execute
    if (count % 1000 == 0):
        print(count, "- found", hotspot_id, "so far.")
print("Found", hotspot_id-1, " hotspots.")


1000 - found 157 so far.
2000 - found 322 so far.
3000 - found 474 so far.
4000 - found 590 so far.
5000 - found 727 so far.
6000 - found 863 so far.
7000 - found 984 so far.
8000 - found 1115 so far.
9000 - found 1225 so far.
10000 - found 1328 so far.
11000 - found 1412 so far.
12000 - found 1500 so far.
13000 - found 1591 so far.
14000 - found 1687 so far.
15000 - found 1768 so far.
16000 - found 1842 so far.
17000 - found 1921 so far.
18000 - found 1988 so far.
19000 - found 2063 so far.
20000 - found 2134 so far.
21000 - found 2197 so far.
22000 - found 2246 so far.
23000 - found 2298 so far.
24000 - found 2342 so far.
25000 - found 2379 so far.
26000 - found 2441 so far.
Cluster of 2 with 2 accidents clashes - with 2434, so now 3 accidents
27000 - found 2474 so far.
Cluster of 3 with 4 accidents clashes - with 2495,2495, so now 5 accidents
28000 - found 2503 so far.
29000 - found 2534 so far.
30000 - found 2568 so far.
Cluster of 3 with 2 accidents clashes - with 2605, so now 4 a

In [ ]:
# Create index for easy display in PowerBI map graphic
hotspots = pd.DataFrame()
for i in range(1,hotspot_id):
    hs = df[df["Hotspot_Index"] == i]
    hotspots = hotspots.append({"Hotspot_Index": i
                    ,"Accident_Count": hs.shape[0]
                   ,"Longitude ": hs["Longitude"].mean()
                   ,"Latitude": hs["Latitude"].mean()
                   }, ignore_index = True)

In [ ]:
# Save our results
df.to_csv("accident_list.csv", index=False)
hotspots.to_csv("hotspot_list.csv", index=False)
